In [1]:
import tensorflow as tf
import numpy as np
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [2]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [3]:
node3 = tf.add(node1, node2)
print("node3:", node3)
#print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)


In [4]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


Have to run the session before anything happens

In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [6]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))


7.5
[ 3.  7.]


In [7]:
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))


[ 3.  7.]


In [12]:
print(sess.run(adder_node, {a: ([0, 1, 1, 3,3, 1, 3, 0]), b: [2, 3, 4, 1, 4, 1, 0, 0]}))



[ 2.  4.  5.  4.  7.  2.  3.  0.]



Can do this with arrays as well

In [22]:
print(sess.run(adder_node, {a: [[0, 1, 1, 3],[3, 1, 3, 0]], b: [[2, 3, 4, 1], [4, 1, 0, 0]]}))



[[ 2.  4.  5.  4.]
 [ 7.  2.  3.  0.]]


In [23]:
import numpy.random as nprnd
g = nprnd.randint(1000, size=10)
print(g)

[ 80 564 290 563  77  38 885 333 715 393]


Add another node


In [24]:

add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [25]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:



In [26]:
init = tf.global_variables_initializer()
sess.run(init)

In [34]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))
print(sess.run({'W': W}))

print(sess.run({'b': b}))




[ 0.          0.30000001  0.60000002  0.90000004]
{'W': array([ 0.30000001], dtype=float32)}
{'b': array([-0.30000001], dtype=float32)}


In [35]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. 
A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. 

We can change W and b accordingly:



In [37]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. 
The simplest optimizer is **gradient descent**. 

It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. 

TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients. For simplicity, optimizers typically do this for you. For example,



In [38]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [39]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## Complete program for Linear Regression Training

In [40]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


Reference:
https://www.tensorflow.org/get_started/get_started

## tf.estimator

** tf.estimator ** is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

running training loops

running evaluation loops

managing data sets


tf.estimator defines many common models.

In [45]:


import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
%matplotlib inline


# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/66/7txfr8_j2wn5skfx0jbnm3300000gn/T/tmp7u62kacm', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/66/7txfr8_j2wn5skfx0jbnm3300000gn/T/tmp7u62kacm/model.ckpt.
INFO:tensorflow:loss = 9.0, step = 1
INFO:tensorflow:global_step/sec: 512.233
INFO:tensorflow:loss = 0.0800316, step = 101 (0.197 sec)
INFO:tensorflow:global_step/sec: 596.736
INFO:tensorflow:loss = 0.0138438, step = 201 (0.166 sec)
INFO:tensorflow:global_step/sec: 632.136
INFO:tensorflow:loss = 0.00035158, step = 301 (0.158 sec)
INFO:tensorflow:global_step/sec: 834.242
INFO:tensorflow:loss = 0.000103384, step = 401 (0.121 sec)
INFO:tensorflow:globa

Will work on making plots later

plt.plot(np.squeeze(costs))

plt.ylabel('cost')

plt.xlabel('iterations (per tens)')

plt.title("Learning rate =" + str(learning_rate))

plt.show()

## Making a Custom Model using lower level functions of tf.estimator


In [46]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/66/7txfr8_j2wn5skfx0jbnm3300000gn/T/tmpiuabtvxt', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/66/7txfr8_j2wn5skfx0jbnm3300000gn/T/tmpiuabtvxt/model.ckpt.
INFO:tensorflow:loss = 9.85315286822, step = 1
INFO:tensorflow:global_step/sec: 1161.94
INFO:tensorflow:loss = 0.0235151029472, step = 101 (0.088 sec)
INFO:tensorflow:global_step/sec: 1205.15
INFO:tensorflow:loss = 0.00330104600997, step = 201 (0.084 sec)
INFO:tensorflow:global_step/sec: 1029.22
INFO:tensorflow:loss = 0.00022648098187, step = 301 (0.096 sec)
INFO:tensorflow:global_step/sec: 1122.56
INFO:tensorflow:loss = 3.30449501539e-05, step = 40

# MINST data

In [47]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Softmax reference
http://neuralnetworksanddeeplearning.com/chap3.html#softmax

In [49]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 784])
#x isn't a specific value. 
#It's a placeholder, a value that we'll input when we ask TensorFlow to run a computation. 
#We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. 
#We represent this as a 2-D tensor of floating-point numbers, with a shape [None, 784]. 
#(Here None means that a dimension can be of any length.)



# what is the difference between a placeholder and a variable?

We also need the weights and biases for our model. We could imagine treating these like additional inputs, but TensorFlow has an even better way to handle it: Variable. 

A **Variable** is a modifiable tensor that lives in TensorFlow's graph of interacting operations. It can be used and even modified by the computation. For machine learning applications, one generally has the model parameters be Variables.


(So is it that a **variable** gets modified, as in forward/backward propagation while a **placeholder** will just hold the value of X)


In [50]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Notice that W has a shape of [784, 10] because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. b has a shape of [10] so we can add it to the output.



In [51]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

# cross-entropy
https://colah.github.io/posts/2015-09-Visual-Information

In [52]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [53]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

Note that in the source code, we don't use this formulation, 

because it is numerically unstable. 

Instead, we apply tf.nn.softmax_cross_entropy_with_logits on the unnormalized logits (e.g., we call softmax_cross_entropy_with_logits on tf.matmul(x, W) + b), because this more numerically stable function internally computes the softmax activation. 

In your code, consider using **tf.nn.softmax_cross_entropy_with_logits** instead.

good reference for back-prop: http://colah.github.io/posts/2015-08-Backprop/

In [54]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

## Optimizers
https://www.tensorflow.org/api_guides/python/train#Optimizers

The Optimizer base class provides methods to compute gradients for a loss and apply gradients to variables. A collection of subclasses implement classic optimization algorithms such as GradientDescent and Adagrad.

You never instantiate the Optimizer class itself, but instead instantiate one of the subclasses.

tf.train.Optimizer

tf.train.GradientDescentOptimizer

tf.train.AdadeltaOptimizer

tf.train.AdagradOptimizer

tf.train.AdagradDAOptimizer

tf.train.MomentumOptimizer

tf.train.AdamOptimizer

tf.train.FtrlOptimizer

tf.train.ProximalGradientDescentOptimizer

tf.train.ProximalAdagradOptimizer

tf.train.RMSPropOptimizer


In [55]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    
    

## Evaluating Our Model


Well, first let's figure out where we predicted the correct label. 

tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. 

For example, tf.argmax(y,1) is the label our model thinks is most likely for each input, while tf.argmax(y_,1) is the correct label. 

We can use tf.equal to check if our prediction matches the truth.

In [56]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

which gives us a list of Boolean values (did our prediction match the correct label)

In [58]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [59]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9111
